In [1]:
print("hello world")

hello world


## Checking Action Space

In [18]:
from aicons.definitions.aicon_types import BadAIcon
import inspect

# Create a BadAIcon
bad_aicon = BadAIcon(
    name="EmptyBrainDemo",
    capabilities=["budget_allocation"]
)

# Print all attributes and their values
print("BadAIcon attributes:")
for attr_name in dir(bad_aicon):
    if not attr_name.startswith('__'):
        try:
            attr_value = getattr(bad_aicon, attr_name)
            if not callable(attr_value):
                print(f"- {attr_name}: {attr_value}")
        except Exception as e:
            print(f"- {attr_name}: Error accessing attribute: {e}")

# Check if brain is in the __dict__
print("\nChecking __dict__:")
for key, value in bad_aicon.__dict__.items():
    print(f"- {key}: {value}")

BadAIcon attributes:
- _abc_impl: <_abc._abc_data object at 0x13d956f40>
- action_space: None
- bayes_brain_config: {'roi': {'type': 'continuous', 'value': 1.0, 'description': 'Return on Investment'}, 'ctr': {'type': 'continuous', 'value': 0.02, 'description': 'Click-through Rate'}, 'cpc': {'type': 'continuous', 'value': 1.0, 'description': 'Cost Per Click'}, 'conversion_rate': {'type': 'continuous', 'value': 0.05, 'description': 'Conversion Rate'}}
- budget_increment: 100.0
- campaigns: {}
- capabilities: ['budget_allocation']
- meta_campaign: None
- name: EmptyBrainDemo
- num_posterior_samples: 1000
- posterior_samples: {'phi': None, 'c': None, 'delta': None}
- state: None
- type: bad

Checking __dict__:
- name: EmptyBrainDemo
- type: bad
- capabilities: ['budget_allocation']
- campaigns: {}
- meta_campaign: None
- state: None
- bayes_brain_config: {'roi': {'type': 'continuous', 'value': 1.0, 'description': 'Return on Investment'}, 'ctr': {'type': 'continuous', 'value': 0.02, 'descri